<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages</a></span></li><li><span><a href="#Data-Processing" data-toc-modified-id="Data-Processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Processing</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#24MONTH-NO-CAP" data-toc-modified-id="24MONTH-NO-CAP-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>24MONTH NO CAP</a></span></li><li><span><a href="#24MONTH-WITH-CAP" data-toc-modified-id="24MONTH-WITH-CAP-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>24MONTH WITH CAP</a></span></li><li><span><a href="#36MONTH-NO-CAP" data-toc-modified-id="36MONTH-NO-CAP-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>36MONTH NO CAP</a></span></li><li><span><a href="#36MONTH-WITH-CAP" data-toc-modified-id="36MONTH-WITH-CAP-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>36MONTH WITH CAP</a></span></li></ul></li><li><span><a href="#Test-Result-Predictions" data-toc-modified-id="Test-Result-Predictions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test Result Predictions</a></span></li></ul></div>

# Import Packages

In [4]:
from arch import arch_model

In [1]:
# Method #1 GARCH Model

import sys, os
sys.path.append('../python/')

from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
from set_params import func_garch_train_test_split, count_train_test
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import warnings

def calculate_iqr(values):
    # Calculate Q1
    Q1 = np.percentile(values, 25)
    # Calculate Q3
    Q3 = np.percentile(values, 75)
    # Calculate IQR
    IQR = Q3 - Q1
    return IQR

def detect_outliers_iqr(values):
    # Calculate the IQR of the values
    IQR = calculate_iqr(values)
    # Calculate Q1 and Q3
    Q1 = np.percentile(values, 25)
    Q3 = np.percentile(values, 75)
    # Define the lower and upper bound for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Return a boolean array: True if the value is an outlier, False otherwise
    return lower_bound, upper_bound


# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

# Data Processing

In [3]:
base_FTSE_df = pd.read_csv('../data/1.3-FTSE_Monthly_ESG_Volatility_Final_v2.csv')
base_FTSE_df = base_FTSE_df.rename(columns={'Date_x':'date_key'})

In [4]:
train_df, valid_df, test_df = func_garch_train_test_split(validation = False, 
                                                    threshold = 24)

In [5]:
count_rows_df = count_train_test(train_df, test_df)

In [6]:
coverage_df = pd.read_csv('../data/coverage_dataframe.csv')
coverage_df.PermID = coverage_df.PermID.astype(int)
coverage_df = coverage_df[['PermID', 'Name']]
coverage_df = coverage_df.rename(columns={'PermID':'Asset'})

In [7]:
train_df = pd.merge(train_df, coverage_df, how = 'left', on = 'Asset')
train_df.index = train_df.month_key

In [8]:
train_df.head(2)

,Unnamed: 0,date_key,Asset,Open,High,Low,Close,Return,V^CC,V^RS,V^YZ,month_key,buzz,ESG,ESGCombined,ESGControversies,EnvironmentalPillar,GovernancePillar,SocialPillar,Community,EnvironmentalInnovation,Management,ProductResponsibility,Shareholders,Workforce,Name
month_key,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-01,0,2020-10-30,5042941681,4.933999,4.935000,4.830999,4.847998,-0.018027,0.015260,0.014832,0.016693,2020-09-01,4108.0,75.0,75.0,75.0,64.0,71.0,91.0,100.0,69.0,91.0,97.0,3.0,81.0,B&M European Value Retail SA
2020-10-01,1,2020-11-30,5042941681,4.742000,4.824998,4.737000,4.778999,0.006105,0.028225,0.025393,0.027841,2020-10-01,4090.5,71.0,71.0,71.0,55.0,69.0,89.0,100.0,66.0,88.0,98.0,4.0,74.0,B&M European Value Retail SA


# Modeling

In [9]:
def compile_train_test_garch(train_df, test_df, sample = True, cap = False, viz = False):
    '''
    '''

    mresults = pd.DataFrame()
    q, p = 1,1
    
    if sample:
        # assets = [4295894970, 8589934212]
        assets = [8589934212, 8589934254] # 3rd and 4th most volatile stocks in FTSE 2006 to 2022
    else:
        assets = train_df.Asset.unique().tolist()

    for r, asset in enumerate(assets): 
        
        rolling_predictions = []
        test_size = test_df[test_df['Asset'] == asset].shape[0]
        
        y_volatility = train_df[train_df['Asset'] == asset]['V^YZ']
        y_volatility.append(test_df[test_df['Asset'] == asset]['V^YZ'])
        y_test = y_volatility[-test_size:]
        name = train_df[train_df['Asset'] == asset].iloc[0,-1]
        
        for i in range(test_size):
            # train data
            y_train = y_volatility[:-(test_size-i)]
            model = arch_model(y_train, p=p, q=q)
            model_fit = model.fit(disp='off')
            # test data
            pred = model_fit.forecast(horizon=1)
            pred = np.sqrt(pred.variance.values[-1,:][0])
            
            # capping the outliers
            if cap:
                lower_bound, upper_bound = detect_outliers_iqr(y_train)
                if upper_bound < pred:
                    pred = upper_bound
            
            rolling_predictions.append(pred)

        indices = test_df[test_df.Asset == asset].index
        rolling_predictions = pd.Series(rolling_predictions, index=indices)
        # print(indices)
        
        mse_million = mean_squared_error(y_test,rolling_predictions)*10**3
        mresult = pd.DataFrame({
            'Asset': asset,
            'Name': name,
            'Model':'GARCH(1,1)',
            'Test Size': test_size,
            'MSE^3':mse_million
                    }
            , index=[r]
        )
        mresults = pd.concat([mresults, mresult])
        
        if viz:
            plt.figure(figsize=(10,4))
            true, = plt.plot(test_df[test_df.Asset == asset]['V^YZ'])
            preds, = plt.plot(rolling_predictions)
            plt.title(f'GARCH(1,1) {name}', fontsize=15)
            plt.legend(['True Volatility', 'Predicted Volatility'], fontsize=9)
            plt.xticks(rotation=45)
            plt.savefig(f'../Outputs/Garch/{str(r+1).zfill(3)}_GARCH-{name}.png')
            plt.close()
        
    return mresults

In [10]:
mresults = compile_train_test_garch(train_df, test_df, sample=False, cap = False, viz = False)

/Users/yogi/miniconda3/envs/nlp2/lib/python3.11/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Users/yogi/miniconda3/envs/nlp2/lib/python3.11/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Users/yogi/miniconda3/envs/nlp2/lib/python3.11/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Users/yogi/miniconda3/envs/nlp2/lib/python3.11/site-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for line

---

# Results

## 24MONTH NO CAP

In [11]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
57,4295894970,Bunzl plc,"GARCH(1,1)",50,81.957906
16,4295897861,Micro Focus International Ltd,"GARCH(1,1)",10,2.096486
74,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,0.990881
37,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
114,4295894987,British Airways PLC,"GARCH(1,1)",17,0.801034


In [11]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
57,4295894970,Bunzl plc,"GARCH(1,1)",50,566538.205193
16,4295897861,Micro Focus International Ltd,"GARCH(1,1)",10,2.096690
74,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,1.030409
37,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.951982
114,4295894987,British Airways PLC,"GARCH(1,1)",17,0.789621


In [13]:
np.mean(mresults['MSE^3'])

0.7918882694578178

In [12]:
np.mean(mresults['MSE^3'])

3934.5152731160906

## 24MONTH WITH CAP

In [20]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
18,4295897861,Micro Focus International Ltd,"GARCH(1,1)",10,1.782181
79,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,1.020862
41,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
128,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
31,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230


In [21]:
np.mean(mresults['MSE^3'])

0.22413324492142586

## 36MONTH NO CAP

In [37]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
51,4295894970,Bunzl plc,"GARCH(1,1)",51,80.352324
67,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,0.990881
31,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
111,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
22,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230


In [38]:
np.mean(mresults['MSE^3'])

0.7667764680687831

## 36MONTH WITH CAP

In [33]:
mresults.sort_values('MSE^3', ascending=False).head(5)

,Asset,Name,Model,Test Size,MSE^3
67,4295895061,Sibanye Uk Ltd,"GARCH(1,1)",18,1.020862
31,5036227579,EVRAZ plc,"GARCH(1,1)",19,0.950165
111,4295894987,British Airways PLC,"GARCH(1,1)",17,0.823081
22,4295898044,Ashtead Group PLC,"GARCH(1,1)",32,0.758230
78,5000047647,Friends Life FPG Ltd,"GARCH(1,1)",12,0.713776


In [34]:
np.mean(mresults['MSE^3'])

0.2017985825595635

---

In [39]:
threshold = str(36)
CAP = 'NCAP'
mresults.to_excel(f'../Results/1-GARCH-{threshold}MONTH-{CAP}.xlsx', index=None)

---

# Test Result Predictions

In [15]:
def compile_train_test_garch(train_df, test_df, sample = True, cap = False, viz = False):
    '''
    '''

    mresults = pd.DataFrame()
    predictions_df = pd.DataFrame()
    q, p = 1,1
    
    if sample:
        # assets = [4295894970, 8589934212]
        assets = [8589934212, 8589934254] # 3rd and 4th most volatile stocks in FTSE 2006 to 2022
    else:
        assets = train_df.Asset.unique().tolist()

    for r, asset in enumerate(assets): 
        
        rolling_predictions = []
        test_size = test_df[test_df['Asset'] == asset].shape[0]
        
        y_volatility = train_df[train_df['Asset'] == asset]['V^YZ']
        y_volatility.append(test_df[test_df['Asset'] == asset]['V^YZ'])
        y_test = y_volatility[-test_size:]
        name = train_df[train_df['Asset'] == asset].iloc[0,-1]
        
        for i in range(test_size):
            # train data
            y_train = y_volatility[:-(test_size-i)]
            model = arch_model(y_train, p=p, q=q)
            model_fit = model.fit(disp='off')
            # test data
            pred = model_fit.forecast(horizon=1)
            pred = np.sqrt(pred.variance.values[-1,:][0])
            
            # capping the outliers
            if cap:
                lower_bound, upper_bound = detect_outliers_iqr(y_train)
                if upper_bound < pred:
                    pred = upper_bound
            
            rolling_predictions.append(pred)

        
        prediction_df = pd.DataFrame({
            'Asset':asset,
            'Test Data': y_test,
            'Predictions': rolling_predictions
        }, index = y_test.index)
        
        
        indices = test_df[test_df.Asset == asset].index
        rolling_predictions = pd.Series(rolling_predictions, index=indices)
        # print(indices)
        
        mse_million = mean_squared_error(y_test,rolling_predictions)*10**3
        mresult = pd.DataFrame({
            'Asset': asset,
            'Name': name,
            'Model':'GARCH(1,1)',
            'Test Size': test_size,
            'MSE^3':mse_million
                    }
            , index=[r]
        )
        
        predictions_df = pd.concat([predictions_df, prediction_df])
        mresults = pd.concat([mresults, mresult])
        
        if viz:
            plt.figure(figsize=(10,4))
            true, = plt.plot(test_df[test_df.Asset == asset]['V^YZ'])
            preds, = plt.plot(rolling_predictions)
            plt.title(f'GARCH(1,1) {name}', fontsize=15)
            plt.legend(['True Volatility', 'Predicted Volatility'], fontsize=9)
            plt.xticks(rotation=45)
            plt.savefig(f'../Outputs/Garch/{str(r+1).zfill(3)}_GARCH-{name}.png')
            plt.close()
        
    return mresults, predictions_df

In [16]:
mresults, pred = compile_train_test_garch(train_df, test_df, sample=True, cap = False, viz = False)

In [20]:
mresults

,Asset,Name,Model,Test Size,MSE^3
0,8589934212,Natwest Group PLC,"GARCH(1,1)",60,0.246405
1,8589934254,Lloyds Banking Group PLC,"GARCH(1,1)",60,0.150474


In [18]:
pred.to_csv('../data/9-GARCH-SAMPLE-PREDICTION-RESULTS.csv')